##第2章: UNIXコマンド
popular-names.txtは，アメリカで生まれた赤ちゃんの「名前」「性別」「人数」「年」をタブ区切り形式で格納したファイルである．以下の処理を行うプログラムを作成し，popular-names.txtを入力ファイルとして実行せよ．さらに，同様の処理をUNIXコマンドでも実行し，プログラムの実行結果を確認せよ．

指定のデータをダウンロードします。
Google Colaboratoryのセル上で下記のコマンドを実行すると、カレントディレクトリに対象のテキストファイルがダウンロードされます。

In [1]:
!wget https://nlp100.github.io/data/popular-names.txt

--2021-01-15 03:02:54--  https://nlp100.github.io/data/popular-names.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55026 (54K) [text/plain]
Saving to: ‘popular-names.txt’

popular-names.txt   100%[===================>]  53.74K  --.-KB/s    in 0.007s  

2021-01-15 03:02:54 (8.05 MB/s) - ‘popular-names.txt’ saved [55026/55026]



####10. 行数のカウント
行数をカウントせよ．確認にはwcコマンドを用いよ．

In [2]:
import pandas as pd

df = pd.read_table('./popular-names.txt', header=None, sep='\t', names=['name', 'sex', 'number', 'year'])
print(len(df))

2780


In [3]:
# 確認
!wc -l ./popular-names.txt

2780 ./popular-names.txt


####11. タブをスペースに置換
タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．

In [4]:
# 確認
!sed -e 's/\t/ /g' ./popular-names.txt | head -n 5

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880


####12. 1列目をcol1.txtに，2列目をcol2.txtに保存
各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．



In [5]:
col1 = df['name']
col1.to_csv('./col1.txt', index=False)
print(col1.head())

0         Mary
1         Anna
2         Emma
3    Elizabeth
4       Minnie
Name: name, dtype: object


In [6]:
# 確認
!cut -f 1 ./popular-names.txt > ./col1_chk.txt
!cat ./col1_chk.txt | head -n 5

Mary
Anna
Emma
Elizabeth
Minnie


In [8]:
col2 = df['sex']
col2.to_csv('./col2.txt', index=False)

In [9]:
# 確認
!cut -f 2 ./popular-names.txt > ./col2_chk.txt
!cat ./col2_chk.txt | head -n 5

F
F
F
F
F


####13. col1.txtとcol2.txtをマージ
12で作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [10]:
col1 = pd.read_table('./col1.txt')
col2 = pd.read_table('./col2.txt')
merged_1_2 = pd.concat([col1, col2], axis=1)
merged_1_2.to_csv('./merged_1_2.txt', sep='\t', index=False)
print(merged_1_2.head())

        name sex
0       Mary   F
1       Anna   F
2       Emma   F
3  Elizabeth   F
4     Minnie   F


In [11]:
# 確認
!paste ./col1_chk.txt ./col2_chk.txt | head -n 5

Mary	F
Anna	F
Emma	F
Elizabeth	F
Minnie	F


####14. 先頭からN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．

In [12]:
def output_head(N):
  print(df.head(N))

output_head(5)

        name sex  number  year
0       Mary   F    7065  1880
1       Anna   F    2604  1880
2       Emma   F    2003  1880
3  Elizabeth   F    1939  1880
4     Minnie   F    1746  1880


In [13]:
# 確認
!head -n 5 ./popular-names.txt

Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880
Elizabeth	F	1939	1880
Minnie	F	1746	1880


####15. 末尾のN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ．

In [14]:
def output_tail(N):
  print(df.tail(N))

output_tail(5)

          name sex  number  year
2775  Benjamin   M   13381  2018
2776    Elijah   M   12886  2018
2777     Lucas   M   12585  2018
2778     Mason   M   12435  2018
2779     Logan   M   12352  2018


In [15]:
# 確認
!tail -n 5 ./popular-names.txt

Benjamin	M	13381	2018
Elijah	M	12886	2018
Lucas	M	12585	2018
Mason	M	12435	2018
Logan	M	12352	2018


####16. ファイルをN分割する
自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN分割せよ．同様の処理をsplitコマンドで実現せよ．

いろいろなやり方があると思いますが、ここではレコードの通番に対して、N分位点を算出するqcutを適用することでファイルをN分割するフラグを付与しています。

In [16]:
def split_file(N):
  tmp = df.reset_index(drop=False)
  df_cut = pd.qcut(tmp.index, N, labels=[i for i in range(N)])
  df_cut = pd.concat([df, pd.Series(df_cut, name='sp')], axis=1)

  return df_cut

df_cut = split_file(10)
print(df_cut['sp'].value_counts())

9    278
8    278
7    278
6    278
5    278
4    278
3    278
2    278
1    278
0    278
Name: sp, dtype: int64


In [17]:
print(df_cut.head())

        name sex  number  year sp
0       Mary   F    7065  1880  0
1       Anna   F    2604  1880  0
2       Emma   F    2003  1880  0
3  Elizabeth   F    1939  1880  0
4     Minnie   F    1746  1880  0


In [18]:
# コマンドによる分割
!split -l 200 -d ./popular-names.txt sp

####17. １列目の文字列の異なり
1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはcut, sort, uniqコマンドを用いよ．

In [19]:
print(len(df.drop_duplicates(subset='name')))

136


In [20]:
# 確認
!cut -f 1 ./popular-names.txt | sort | uniq | wc -l

136


####18. 各行を3コラム目の数値の降順にソート
各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．

In [21]:
df.sort_values(by='number', ascending=False, inplace=True)
print(df.head())

         name sex  number  year
1340    Linda   F   99689  1947
1360    Linda   F   96211  1948
1350    James   M   94757  1947
1550  Michael   M   92704  1957
1351   Robert   M   91640  1947


In [22]:
# 確認
!cat ./popular-names.txt | sort -rnk 3 | head -n 5

Linda	F	99689	1947
Linda	F	96211	1948
James	M	94757	1947
Michael	M	92704	1957
Robert	M	91640	1947


####19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．

In [23]:
print(df['name'].value_counts())

James      118
William    111
Robert     108
John       108
Mary        92
          ... 
Laura        1
Scott        1
Lori         1
Julie        1
Crystal      1
Name: name, Length: 136, dtype: int64


In [25]:
# 確認
!cut -f 1 ./popular-names.txt | sort | uniq -c | sort -rn

    118 James
    111 William
    108 Robert
    108 John
     92 Mary
     75 Charles
     74 Michael
     73 Elizabeth
     70 Joseph
     60 Margaret
     58 Thomas
     58 George
     57 David
     51 Richard
     45 Helen
     43 Frank
     43 Christopher
     41 Anna
     40 Edward
     39 Ruth
     38 Patricia
     37 Matthew
     36 Dorothy
     35 Emma
     32 Barbara
     31 Joshua
     31 Daniel
     26 Sarah
     26 Linda
     26 Jennifer
     26 Emily
     25 Jessica
     25 Jacob
     24 Susan
     24 Mildred
     24 Betty
     23 Henry
     23 Ashley
     22 Nancy
     21 Andrew
     20 Marie
     20 Florence
     20 Donald
     20 Amanda
     19 Samantha
     18 Olivia
     18 Melissa
     18 Madison
     18 Lisa
     18 Karen
     17 Stephanie
     17 Abigail
     16 Sandra
     16 Mark
     16 Ethel
     15 Michelle
     15 Isabella
     15 Heather
     15 Frances
     15 Ethan
     15 Carol
     15 Angela
     14 Shirley
     14 Kimberly
     14 Ava
     14 Amy
     